# Problema 1

Considere un juego de disparejo entre cuatro jugadores, en el que cada jugador adopta la siguiente estrategia:

    - El jugador 1 siempre escoge al azar entre las dos opciones.
    - El jugador 2 escoge siempre hacia abajo sin importar lo que haya ocurrido anteriormente.
    - El jugador 3 escoge la última opción ganadora de las partidas anteriores. En la primera jugada, escoge arriba.
    - El jugador 4 escoge aquello opuesto al jugador 1 en la última partida. En la primera jugada, escoge al azar.

Utilice AgentPy para generar una simulación del juego, y simule 1000 juegos consecutivos. Muestre los resultados obtenidos para cada uno de los jugadores al final de la simulación.

¿Qué jugador obtuvo mejores resultados? ¿Alguna de las estrategias fue mejor que las otras?


In [1]:
#Model
import agentpy as ap

# Visualization
import seaborn as sns
import IPython

# Native
from enum import Enum
import random

#### Agent definition

In [2]:
class DIRECTION(Enum):
    DOWN = 0
    UP = 1

class Player(ap.Agent):

    def setup(self):
        self.prev_direction = None
        self.curr_direction = None
        self.score = 0

    def newCurrDirection(self):
        self.prev_direction = self.curr_direction

        match self.id:
            case 1:
                self.curr_direction = random.randint(0, 1)
            case 2:
                self.curr_direction = DIRECTION.DOWN.value
            case 3:
                if self.model.last_player_won_id is not None:
                    prev_user_dir = self.model.agents[self.model.last_player_won_id - 1].prev_direction
                    self.curr_direction = prev_user_dir
                elif self.model.t == 0:
                    self.curr_direction = DIRECTION.UP.value
            case 4:
                player_1_prev_direction = self.model.agents[0].prev_direction

                if player_1_prev_direction is not None:
                    if player_1_prev_direction == DIRECTION.UP.value:
                        self.curr_direction = DIRECTION.DOWN.value
                    else:
                        self.curr_direction = DIRECTION.UP.value
                elif self.model.t == 0:
                    self.curr_direction = random.randint(0, 1)

            case _:
                print("Error: unknown player: ", self.id)


In [3]:
class Model(ap.Model):
    def checkWinner(self):
        zero_count = 0
        self.curr_winner_dir = None

        for agent in self.agents:
            if agent.curr_direction == 0:
                zero_count += 1

        for agent in self.agents:
            if zero_count == 1:
                if agent.curr_direction == 0:
                    agent.score += 1
                    self.last_player_won_id = agent.id
                    break

            elif zero_count == 3:
                if agent.curr_direction == 1:
                    agent.score += 1
                    self.last_player_won_id = agent.id
                    break

    def setup(self):
        self.last_player_won_id = None

        self.agents = ap.AgentList(self, 4, Player)
        self.agents.newCurrDirection()
        self.checkWinner()

    def step(self):
        self.agents.newCurrDirection()
        self.last_player_won_id = None

        self.checkWinner()

    def update(self):
        self.agents.record('curr_direction')
        self.agents.record('score')

In [4]:
parameters = {'steps': 5}
model = Model(parameters)

res = model.run()

Completed: 5 steps
Run time: 0:00:00.003534
Simulation finished


In [5]:
res.variables.Player

curr_direction  score
obj_id t                       
1      0               0      0
       1               1      0
       2               0      0
       3               1      0
       4               1      1
       5               1      1
2      0               0      0
       1               0      1
       2               0      1
       3               0      1
       4               0      1
       5               0      1
3      0               1      1
       1               1      1
       2               0      1
       3               0      1
       4               0      1
       5               1      1
4      0               0      0
       1               1      0
       2               0      0
       3               1      0
       4               0      0
       5               0      0

In [8]:
res.

,seed
0,47781996468612601228286266083587513407
